In [1]:
import numpy as np
import pandas as pd
import gc
import time
from contextlib import contextmanager
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import KFold, StratifiedKFold
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from sklearn.externals import joblib
%matplotlib inline
import seaborn as sn
from tqdm import tqdm_notebook as tqdm
from scipy.stats import skew, kurtosis, iqr

In [2]:
#get top 700 features

In [3]:
pruning=pd.read_csv('feature_importance_lgb1000features_0.01.csv')

In [4]:
pruning

,feature,importance,fold
0,AMT_ANNUITY,4378.567476,1
1,AMT_CREDIT,4510.543404,1
2,AMT_GOODS_PRICE,11125.428929,1
3,AMT_INCOME_TOTAL,1318.231940,1
4,AMT_REQ_CREDIT_BUREAU_DAY,4.483940,1
5,AMT_REQ_CREDIT_BUREAU_HOUR,0.000000,1
6,AMT_REQ_CREDIT_BUREAU_MON,353.941950,1
7,AMT_REQ_CREDIT_BUREAU_QRT,764.060712,1
8,AMT_REQ_CREDIT_BUREAU_WEEK,0.000000,1
9,AMT_REQ_CREDIT_BUREAU_YEAR,313.170028,1


In [5]:
pruning_object=pruning.groupby(by='feature').agg({'importance':sum}).reset_index()

In [6]:
pruning_object

,feature,importance
0,100_period_trend_instalment_paid_late_in_days,8904.774864
1,100_period_trend_instalment_paid_over_amount,4496.638091
2,10_period_trend_instalment_paid_late_in_days,8809.279322
3,10_period_trend_instalment_paid_over_amount,4448.706900
4,12_period_trend_SK_DPD,3754.279314
5,12_period_trend_SK_DPD_DEF,5878.975600
6,1_period_trend_SK_DPD,0.000000
7,1_period_trend_SK_DPD_DEF,0.000000
8,30_period_trend_SK_DPD,4807.559949
9,30_period_trend_SK_DPD_DEF,8371.905733


In [7]:
pruning_rank=pruning_object.sort_values('importance',ascending=False).reset_index()

In [8]:
pruning_rank

,index,feature,importance
0,506,external_sources_mean,404517.568295
1,507,external_sources_min,247947.650197
2,508,external_sources_nanmedian,222004.931958
3,509,external_sources_sum,205494.497796
4,98,EXT_SOURCE_2,187629.437967
5,505,external_sources_max,180590.734657
6,501,credit_to_annuity_ratio,160435.178030
7,1003,payment_rate,153071.092508
8,99,EXT_SOURCE_3,147863.032595
9,510,external_sources_weighted,127387.973797


In [9]:
pruning_rank_700=pruning_rank.loc[0:700,]

In [10]:
pruning_rank_700

,index,feature,importance
0,506,external_sources_mean,404517.568295
1,507,external_sources_min,247947.650197
2,508,external_sources_nanmedian,222004.931958
3,509,external_sources_sum,205494.497796
4,98,EXT_SOURCE_2,187629.437967
5,505,external_sources_max,180590.734657
6,501,credit_to_annuity_ratio,160435.178030
7,1003,payment_rate,153071.092508
8,99,EXT_SOURCE_3,147863.032595
9,510,external_sources_weighted,127387.973797


In [11]:
pruning_rank_700_name=np.array(pruning_rank_700['feature'])

In [12]:
pruning_rank_700_name

array(['external_sources_mean', 'external_sources_min',
       'external_sources_nanmedian', 'external_sources_sum',
       'EXT_SOURCE_2', 'external_sources_max', 'credit_to_annuity_ratio',
       'payment_rate', 'EXT_SOURCE_3', 'external_sources_weighted',
       'ORGANIZATION_TYPE', 'OCCUPATION_TYPE_mean_EXT_SOURCE_3_diff',
       'NAME_EDUCATION_TYPE_OCCUPATION_TYPE_mean_EXT_SOURCE_3_diff',
       'credit_to_goods_ratio',
       'NAME_EDUCATION_TYPE_OCCUPATION_TYPE_mean_EXT_SOURCE_2_diff',
       'SK_ID_CURR_max_DAYS_CREDIT',
       'CODE_GENDER_NAME_EDUCATION_TYPE_OCCUPATION_TYPE_REG_CITY_NOT_WORK_CITY_mean_EXT_SOURCE_2_diff',
       'DAYS_EMPLOYED',
       'NAME_EDUCATION_TYPE_OCCUPATION_TYPE_mean_EXT_SOURCE_2_abs_diff',
       'CODE_GENDER_NAME_EDUCATION_TYPE_mean_EXT_SOURCE_2_diff',
       'CODE_GENDER_NAME_EDUCATION_TYPE_mean_EXT_SOURCE_1_diff',
       'CODE_GENDER_NAME_EDUCATION_TYPE_mean_EXT_SOURCE_2_abs_diff',
       'OCCUPATION_TYPE_mean_EXT_SOURCE_3_abs_diff', 'OCCUPATION

In [13]:
pruning_rank_700_name=np.append(pruning_rank_700_name,('TARGET','SK_ID_CURR','index'))

In [14]:
pruning_rank_700_name

array(['external_sources_mean', 'external_sources_min',
       'external_sources_nanmedian', 'external_sources_sum',
       'EXT_SOURCE_2', 'external_sources_max', 'credit_to_annuity_ratio',
       'payment_rate', 'EXT_SOURCE_3', 'external_sources_weighted',
       'ORGANIZATION_TYPE', 'OCCUPATION_TYPE_mean_EXT_SOURCE_3_diff',
       'NAME_EDUCATION_TYPE_OCCUPATION_TYPE_mean_EXT_SOURCE_3_diff',
       'credit_to_goods_ratio',
       'NAME_EDUCATION_TYPE_OCCUPATION_TYPE_mean_EXT_SOURCE_2_diff',
       'SK_ID_CURR_max_DAYS_CREDIT',
       'CODE_GENDER_NAME_EDUCATION_TYPE_OCCUPATION_TYPE_REG_CITY_NOT_WORK_CITY_mean_EXT_SOURCE_2_diff',
       'DAYS_EMPLOYED',
       'NAME_EDUCATION_TYPE_OCCUPATION_TYPE_mean_EXT_SOURCE_2_abs_diff',
       'CODE_GENDER_NAME_EDUCATION_TYPE_mean_EXT_SOURCE_2_diff',
       'CODE_GENDER_NAME_EDUCATION_TYPE_mean_EXT_SOURCE_1_diff',
       'CODE_GENDER_NAME_EDUCATION_TYPE_mean_EXT_SOURCE_2_abs_diff',
       'OCCUPATION_TYPE_mean_EXT_SOURCE_3_abs_diff', 'OCCUPATION

In [15]:
type(pruning_rank_700_name)

numpy.ndarray

In [16]:
#read df

In [17]:
df=pd.read_csv('df_1000.csv')

In [18]:
df.shape

(356255, 1022)

In [19]:
import category_encoders as ce

In [20]:
categorical_columns=['CODE_GENDER',
                       'EMERGENCYSTATE_MODE',
                       'FLAG_CONT_MOBILE',
                       'FLAG_DOCUMENT_3',
                       'FLAG_DOCUMENT_4',
                       'FLAG_DOCUMENT_5',
                       'FLAG_DOCUMENT_6',
                       'FLAG_DOCUMENT_7',
                       'FLAG_DOCUMENT_8',
                       'FLAG_DOCUMENT_9',
                       'FLAG_DOCUMENT_11',
                       'FLAG_DOCUMENT_18',
                       'FLAG_EMAIL',
                       'FLAG_EMP_PHONE',
                       'FLAG_MOBIL',
                       'FLAG_OWN_CAR',
                       'FLAG_OWN_REALTY',
                       'FLAG_PHONE',
                       'FLAG_WORK_PHONE',
                       'FONDKAPREMONT_MODE',
                       'HOUR_APPR_PROCESS_START',
                       'HOUSETYPE_MODE',
                       'LIVE_CITY_NOT_WORK_CITY',
                       'LIVE_REGION_NOT_WORK_REGION',
                       'NAME_CONTRACT_TYPE',
                       'NAME_TYPE_SUITE',
                       'NAME_INCOME_TYPE',
                       'NAME_EDUCATION_TYPE',
                       'NAME_FAMILY_STATUS',
                       'NAME_HOUSING_TYPE',
                       'OCCUPATION_TYPE',
                       'ORGANIZATION_TYPE',
                       'REG_CITY_NOT_LIVE_CITY',
                       'REG_CITY_NOT_WORK_CITY',
                       'REG_REGION_NOT_LIVE_REGION',
                       'REG_REGION_NOT_WORK_REGION',
                       'WALLSMATERIAL_MODE',
                       'WEEKDAY_APPR_PROCESS_START']

In [21]:
type(categorical_columns)

list

In [22]:
for col in categorical_columns:
    df[col] = df[col].astype('category')

In [23]:
for col in categorical_columns:
    df[col] = pd.Categorical(df[col].cat.codes+1)

In [24]:
df[categorical_columns].head()

,CODE_GENDER,EMERGENCYSTATE_MODE,FLAG_CONT_MOBILE,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,...,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,OCCUPATION_TYPE,ORGANIZATION_TYPE,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,WALLSMATERIAL_MODE,WEEKDAY_APPR_PROCESS_START
0,2,1,2,2,1,1,1,1,1,1,...,4,2,9,6,1,1,1,1,6,7
1,1,1,2,2,1,1,1,1,1,1,...,2,2,4,40,1,1,1,1,1,2
2,2,0,2,1,1,1,1,1,1,1,...,4,2,9,12,1,1,1,1,0,2
3,1,0,2,2,1,1,1,1,1,1,...,1,2,9,6,1,1,1,1,0,7
4,2,0,2,1,1,1,1,1,2,1,...,4,2,4,38,1,2,1,1,0,5


In [25]:
for col in categorical_columns:
    df[col] = df[col].astype(int)

In [26]:
df.shape

(356255, 1022)

In [27]:
df[categorical_columns].dtypes

CODE_GENDER                    int32
EMERGENCYSTATE_MODE            int32
FLAG_CONT_MOBILE               int32
FLAG_DOCUMENT_3                int32
FLAG_DOCUMENT_4                int32
FLAG_DOCUMENT_5                int32
FLAG_DOCUMENT_6                int32
FLAG_DOCUMENT_7                int32
FLAG_DOCUMENT_8                int32
FLAG_DOCUMENT_9                int32
FLAG_DOCUMENT_11               int32
FLAG_DOCUMENT_18               int32
FLAG_EMAIL                     int32
FLAG_EMP_PHONE                 int32
FLAG_MOBIL                     int32
FLAG_OWN_CAR                   int32
FLAG_OWN_REALTY                int32
FLAG_PHONE                     int32
FLAG_WORK_PHONE                int32
FONDKAPREMONT_MODE             int32
HOUR_APPR_PROCESS_START        int32
HOUSETYPE_MODE                 int32
LIVE_CITY_NOT_WORK_CITY        int32
LIVE_REGION_NOT_WORK_REGION    int32
NAME_CONTRACT_TYPE             int32
NAME_TYPE_SUITE                int32
NAME_INCOME_TYPE               int32
N

In [28]:
#get new df

In [29]:
df=df[pruning_rank_700_name]

In [30]:
df.shape

(356255, 704)

In [31]:
df.to_csv('df_700.csv', index=False)